<a href="https://colab.research.google.com/github/alaabakk/E2504_bachelor/blob/main/Models/Training/ModelTrainingOptuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # Mounts Google Drive to access files stored in your Drive

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install ultralytics optuna roboflow # Installs required packages for YOLO training, hyperparameter optimization, and dataset handling

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 142.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211

In [ ]:
!nvidia-smi # Displays GPU information

from ultralytics import YOLO # Imports the YOLO class from the Ultralytics package
import os # Imports the os module for interacting with the operating system
!yolo checks # Runs a check to verify the YOLO environment and dependencies

Tue May 13 11:47:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             42W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
#Dataset downloaded from RoboFlow, this snippet can be found in your RoboFlow user with the correct API_key and project workspace

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("e2504").project("bachelor_project-gpkyt")
version = project.version(6)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Bachelor_project-6 in yolov8:: 100%|██████████| 11239/11239 [00:01<00:00, 7940.15it/s]


In [ ]:
# Imports
import os
import time
import pandas as pd
import optuna
from optuna.samplers import TPESampler
from optuna.pruners  import SuccessiveHalvingPruner
from datetime import datetime as dt
from zoneinfo import ZoneInfo as zi

# Create root directory for this training session
# Output folder is timestamped for uniqueness and traceability
drive_base = "/content/drive/MyDrive/Your_Folder"
os.makedirs(drive_base, exist_ok=True)

timestamp = dt.now(zi("Europe/Oslo")).strftime("%Y%m%d_%H%M")
model_file = "yolov8n.pt"
base_name  = os.path.splitext(os.path.basename(model_file))[0]
root_dir   = os.path.join(drive_base, f"{base_name}_{timestamp}")
os.makedirs(root_dir, exist_ok=True)

print(f"All output will be saved to: {root_dir}")

# Define the Optuna objective function
def objective(trial):
    # 1) Suggest hyperparameters within defined ranges
    lr0          = trial.suggest_float("lr0",          1e-5, 1e-1, log=True)
    momentum     = trial.suggest_float("momentum",     0.6,  0.98)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    optimizer    = trial.suggest_categorical("optimizer", ["SGD","Adam"])
    batch        = trial.suggest_int("batch",           8,   32)

    # 2) Load model and define pruning callback based on validation mAP
    model = YOLO(model_file)
    def prune_callback(trainer):
        epoch = trainer.epoch
        score = trainer.metrics.get("metrics/mAP50-95(B)")
        if score is None:
            return
        trial.report(score, step=epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    model.add_callback("on_fit_epoch_end", prune_callback)

    # 3) Train model using suggested hyperparameters
    run_name = f"training{trial.number}"
    results = model.train(
        data         = "/content/Your_data.yaml_file", #Path to data.yaml file uploaded from RoboFlow
        epochs       = 300,
        imgsz        = 640,
        lr0          = lr0,
        momentum     = momentum,
        weight_decay = weight_decay,
        optimizer    = optimizer,
        batch        = batch,
        project      = root_dir,
        name         = run_name,
        exist_ok     = False,
        verbose      = False
    )

    # 4) Read the results CSV and return the best mAP50-95
    results_csv = os.path.join(results.save_dir, "results.csv")
    for _ in range(5):
        if os.path.exists(results_csv):
            break
        time.sleep(1)
    df = pd.read_csv(results_csv)
    return df["metrics/mAP50-95(B)"].max()

# 5) Run the Optuna hyperparameter study
if __name__ == "__main__":
    sampler = TPESampler()
    pruner  = SuccessiveHalvingPruner()
    db_path = os.path.join(root_dir, f"optuna_study.db")

    study = optuna.create_study(
        study_name = "yolo_study",
        storage    = f"sqlite:///{db_path}",
        sampler    = sampler,
        pruner     = pruner,
        direction  = "maximize",
        load_if_exists = True
    )
    study.optimize(objective, n_trials=100)

    # 6) Save and print results
    best_trial = study.best_trial
    best_training_name = f"training{best_trial.number}"

    print("Best hyperparameters:", best_trial.params)
    print(f"Best mAP50-95: {best_trial.value:.4f}")
    print(f"Best training run: {best_training_name}")

    # Save all trial history to CSV
    study.trials_dataframe().to_csv(os.path.join(root_dir, "optuna_trials.csv"), index=False)
    print(f"Optuna history saved to: {os.path.join(root_dir, 'optuna_trials.csv')}")




All output vil legges i: /content/drive/MyDrive/YoloRuns/yolov8n_20250501_1246


[I 2025-05-01 10:46:26,771] A new study created in RDB with name: yolo_study


100%|██████████| 6.25M/6.25M [00:00<00:00, 338MB/s]


Ultralytics 8.3.122 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Bachelor_project-6/data.yaml, epochs=300, time=None, patience=100, batch=17, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/YoloRuns/yolov8n_20250501_1246, name=training0, exist_ok=False, pretrained=True, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, s

100%|██████████| 5.35M/5.35M [00:00<00:00, 346MB/s]


...

                   all        663       1209      0.894      0.811      0.901      0.635

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      3/300      5.31G      1.275      1.445      1.108          7        640: 100%|██████████| 309/309 [00:28<00:00, 10.95it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:03<00:00,  7.51it/s]


                   all        663       1209      0.928      0.846      0.928       0.68


[I 2025-05-02 02:22:31,575] Trial 99 pruned. 


Beste hyperparametere: {'lr0': 0.00035203425110463815, 'momentum': 0.8898639852161214, 'weight_decay': 0.00010131859380218626, 'optimizer': 'Adam', 'batch': 30}
Beste mAP50-95: 0.9044
Beste trening: training1
Optuna‐historikk lagret som /content/drive/MyDrive/YoloRuns/yolov8n_20250501_1246/optuna_trials.csv
